<h1>Annual Change in Debt Redemption</h1>
Create a bar graph showing the change in the amount of debt redeemed from year to year and month to month 

In [43]:
using CSV, DataFrames, DataFramesMeta, Gadfly, XLSX

In [44]:
function clean_df(df) 
    df[:, :dollars] = replace(df[:, :dollars], "NaN" => missing) # replace "NaN" with missing
    df[:, :cents] = replace(df[:, :cents], "NaN" => missing) # replace "NaN" with missing
    df[:, :dollars] = replace(df[:, :dollars], missing => 0)  # replace missing with 0 
    df[:, :cents] = replace(df[:, :cents], missing => 0)  # replace missing with 0 
    df 
end 

function group_by_year(df) 
    gdf = @by(df, :year,
        :total_amt = sum(:dollars) + sum(:cents)  # calculate total amount of debt 
    )
    sort!(gdf) # sort by year 
    replace!(gdf.year, missing => "missing") # change missing values to string to allow for plotting
    return gdf 
end

#plot debt redeemed per year saved as svg 
function plot_debt(df::DataFrame, state)
    set_default_plot_size(16cm, 9cm)

    p_debt_date = Gadfly.plot(
        df,
        x=:year,
        y=:total_amt,
        Geom.bar,
        Guide.xlabel("Year"),
        Guide.ylabel("Total Debt (in dollars)"),
        Guide.title(uppercase(state)*" Debt Redeemed Per Year"),
        Gadfly.Theme(background_color = "white")
    )

    println(state)
    img = SVG("analysis/debt_per_year/" * uppercase(state) * "_debt_redeemed_per_year.svg", 16cm, 9cm) 
    draw(img, p_debt_date)  
end 

plot_debt (generic function with 1 method)

In [45]:
# import cd_info 
cd_info = DataFrame(CSV.File("analysis/cd_info.csv"))
# store total amount and years of all states in a new dataframe 
all_states = DataFrame([[], []], [:year, :total_amt])

# loop through cd_info per state 
for i in 1:nrow(cd_info)
    state_df = DataFrame() # create new dataframe for each state
    state_row = cd_info[i, :]
    state_excel = XLSX.readxlsx(state_row[:file_path])
    state_sheet = state_excel["Sheet1"]
    
    # get year columns --> merge years columns into one column 
    year_col_indexes = split(state_row[:year_col], ",")
    for year_col_index in year_col_indexes
        year_col_range = year_col_index*string(state_row[:first_row])*":"*year_col_index*string(state_row[:last_row])
        year_col = state_sheet[year_col_range]

        if nrow(state_df) == 0 # if state_df is empty, create year column 
            state_df.year = vec(year_col)
        else 
            state_df.year .= coalesce(state_df.year, vec(year_col)) # append year col to state dataframe
        end 
    end

    state_df.year = string.(state_df.year) # convert year column to string to allow for replacing missing values

    # add dollar amount column to states dataframe 
    dollar_indexes = split(state_row[:dollars_col], ",") 
    for dollar_index in dollar_indexes
        dollar_col_range = dollar_index*string(state_row[:first_row])*":"*dollar_index*string(state_row[:last_row])
        dollar_col = state_sheet[dollar_col_range]
        state_df.dollars = vec(dollar_col)
    end

    # add cents amount column to states dataframe 
    if !ismissing(state_row[:cents_col]) # handle excel spreadsheets with no cents column 
        cents_indexes = split(state_row[:cents_col], ",")
        for cents_index in cents_indexes
            cents_col_range = cents_index*string(state_row[:first_row])*":"*cents_index*string(state_row[:last_row])
            cents_col = state_sheet[cents_col_range]
            state_df.cents = vec(cents_col) ./ 100 # convert cents to decimal
        end 
    else 
        state_df.cents = zeros(nrow(state_df)) # create column of zeros to allow for summing
    end

    # add state label 
    state_df.state = fill(state_row[:state], nrow(state_df))

    state_df_clean = clean_df(state_df) # clean table to remove missing values
    state_gdf = group_by_year(state_df_clean) # group by year and sum debt
    plot_debt(state_gdf, state_row[:state]) # plot debt redeemed per year saved as svg

    all_states = vcat(all_states, state_gdf) # append state dataframe to all_states dataframe

    println(first(state_gdf, 5)) 
end 


ct
5×2 DataFrame
 Row │ year    total_amt    
     │ String  Float64      
─────┼──────────────────────
   1 │ 1790       1.18777e5
   2 │ 1791       2.24218e5
   3 │ 1792    2255.55
   4 │ 1793    3351.04
   5 │ 1794       1.25849e5


ga
5×2 DataFrame
 Row │ year    total_amt 
     │ String  Float64   
─────┼───────────────────
   1 │ 1790      2277.27
   2 │ 1791     95739.3
   3 │ 1792      3967.43
   4 │ 1793      1046.81
   5 │ 1794     24012.9
md


5×2 DataFrame
 Row │ year    total_amt     
     │ String  Float64       
─────┼───────────────────────
   1 │ 1790        1.40061e5
   2 │ 1791        5.04802e5
   3 │ 1792     7006.75
   4 │ 1793    44582.1
   5 │ 1794      717.89
nc
4×2 DataFrame
 Row │ year    total_amt 
     │ String  Float64   
─────┼───────────────────
   1 │ 1791     33666.1
   2 │ 1792     32129.7
   3 │ 1793      4713.45
   4 │ 1794     14401.9
nh
3×2 DataFrame
 Row │ year    total_amt 
     │ String  Float64   
─────┼───────────────────
   1 │ 1791    420772.0
   2 │ 1792     37316.9
   3 │ 1793      4597.44


nj
5×2 DataFrame
 Row │ year    total_amt     
     │ String  Float64       
─────┼───────────────────────
   1 │ 1791        2.35745e5
   2 │ 1792    17893.8
   3 │ 1793     3665.43
   4 │ 1794    16517.0
   5 │ 1795      809.04


ny
5×2 DataFrame
 Row │ year    total_amt 
     │ String  Float64   
─────┼───────────────────
   1 │ 1790    5.4273e5
   2 │ 1791    2.6001e6
   3 │ 1792    7.86956e5
   4 │ 1793    4.24138e5
   5 │ 1794    2.78669e5


pa


5×2 DataFrame
 Row │ year     total_amt     
     │ String   Float64       
─────┼────────────────────────
   1 │ 1790         2.03456e5
   2 │ 1791         6.06171e5
   3 │ 1792     34862.0
   4 │ 1793     16283.8
   5 │ missing  76527.9
ri
5×2 DataFrame
 Row │ year    total_amt     
     │ String  Float64       
─────┼───────────────────────
   1 │ 1790    36863.3
   2 │ 1791        1.46409e5
   3 │ 1792     8258.96
   4 │ 1793     3682.69
   5 │ 1794    60622.3
sc


5×2 DataFrame
 Row │ year    total_amt     
     │ String  Float64       
─────┼───────────────────────
   1 │ 1790    27513.8
   2 │ 1791    78834.3
   3 │ 1792    15058.0
   4 │ 1793     3137.15
   5 │ 1794        2.48973e5
va
5×2 DataFrame
 Row │ year    total_amt     
     │ String  Float64       
─────┼───────────────────────
   1 │ 1790    48870.6
   2 │ 1791        2.99654e5
   3 │ 1792    16614.6
   4 │ 1793     6551.69
   5 │ 1794     3945.98


In [46]:
all_states_gdf = @by(all_states, :year,
    :total_amt = sum(:total_amt)  
) # group by year and sum debt
replace!(all_states_gdf.year, "missing" => missing) # change string to missing to allow for sorting 
sort!(all_states_gdf) # sort by year
replace!(all_states_gdf.year, missing=> "missing") # change missing values to string to allow for plotting
plot_debt(all_states_gdf, "United States") # plot debt redeemed per year saved as svg

United States


false

In [47]:
all_states_gdf

Row,year,total_amt
,Any,Float64
1,1790,1.12055e6
2,1791,5.24611e6
3,1792,9.62319e5
4,1793,5.15749e5
5,1794,7.73709e5
6,1795,1.93738e5
7,1796,7873.3
8,1797,1.45392e5
9,1798,2238.45
